# Export YOLOv8 Model at Multiple Sizes (320, 416, 640)

This notebook exports the trained YOLOv8 model to ONNX format at different input sizes for performance optimization.

## 1. Install Dependencies

In [ ]:
!pip install ultralytics

## 2. Upload Model File

Upload `unified-detection-v7.pt` to Colab using the file browser on the left.

In [ ]:
import os
from ultralytics import YOLO

# Check if model file exists
model_path = 'unified-detection-v7.pt'
if os.path.exists(model_path):
    print(f"✅ Found model file: {model_path}")
    print(f"   Size: {os.path.getsize(model_path) / 1024 / 1024:.2f} MB")
else:
    print("❌ Model file not found!")
    print("Please upload unified-detection-v7.pt to Colab")

## 3. Export Models at Different Sizes

In [ ]:
# Load the model
model = YOLO(model_path)
print("Model loaded successfully!")

# Export at different sizes
sizes = [320, 416, 640]
exported_files = []

for size in sizes:
    print(f"\n{'='*60}")
    print(f"Exporting model at {size}x{size}...")
    
    # Export to ONNX
    onnx_path = model.export(
        format='onnx',
        imgsz=size,
        simplify=True,
        opset=12,  # Compatible with ONNX Runtime Mobile
        batch=1,
        dynamic=False  # Fixed size for mobile optimization
    )
    
    # Rename to include size in filename
    import shutil
    output_name = f'unified-detection-v7-{size}.onnx'
    
    # Handle both Path and string return types
    if hasattr(onnx_path, 'name'):
        shutil.move(str(onnx_path), output_name)
    else:
        shutil.move(onnx_path, output_name)
    
    exported_files.append(output_name)
    
    print(f"✅ Exported: {output_name}")
    print(f"   Size: {os.path.getsize(output_name) / 1024 / 1024:.2f} MB")

## 4. Verify Exported Models

In [ ]:
print("\n🎉 Export Summary:")
print("="*60)

for filename in exported_files:
    if os.path.exists(filename):
        size_mb = os.path.getsize(filename) / 1024 / 1024
        print(f"✅ {filename}: {size_mb:.2f} MB")
    else:
        print(f"❌ {filename}: Not found")

print("\n📥 Download these files and copy them to:")
print("   - android/src/main/assets/")
print("   - example/assets/")

## 5. Create ZIP for Easy Download

In [ ]:
import zipfile

# Create a zip file with all exported models
zip_filename = 'onnx_models_all_sizes.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for filename in exported_files:
        if os.path.exists(filename):
            zipf.write(filename)
            print(f"Added {filename} to zip")

print(f"\n✅ Created {zip_filename}")
print(f"   Size: {os.path.getsize(zip_filename) / 1024 / 1024:.2f} MB")
print("\n📥 Download this zip file for all models at once!")

## 6. Test Model Loading (Optional)

In [ ]:
# Optional: Test that ONNX models can be loaded
try:
    import onnxruntime as ort
    
    for size in sizes:
        model_file = f'unified-detection-v7-{size}.onnx'
        if os.path.exists(model_file):
            session = ort.InferenceSession(model_file)
            input_name = session.get_inputs()[0].name
            input_shape = session.get_inputs()[0].shape
            output_shape = session.get_outputs()[0].shape
            
            print(f"\n{model_file}:")
            print(f"  Input: {input_name} {input_shape}")
            print(f"  Output: {output_shape}")
            print(f"  ✅ Model loads successfully!")
            
except ImportError:
    print("onnxruntime not installed, skipping verification")
except Exception as e:
    print(f"Error testing models: {e}")